In [1]:
from cassie import CassieEnv
import numpy as np

import time

env = CassieEnv("walking")

qpos = np.copy(env.sim.qpos())

jpos = qpos[env.pos_idx]

fbpos = qpos[0:7]

# keep feet from intersecting ground
fbpos[2] += 0.2

In [2]:
# Joint limits converted to radians from XML file

ROLL_LOW   = -0.26
ROLL_HIGH  =  0.39

YAW_LOW    = -0.39
YAW_HIGH   =  0.39

PITCH_LOW  = -0.87
PITCH_HIGH =  1.4

KNEE_LOW   = -2.86
KNEE_HIGH  = -0.64

FOOT_LOW   = -2.44
FOOT_HIGH  = -0.52

# achilles = .75 + pitch = 1
# not currently correct though
ACHILLES_LIMIT = 1.75

DELTA = 0.05#0.025 # resolution for IK data set
ITERS = 100#500

# ramp down joint angles
while jpos[2] > PITCH_LOW and jpos[3] > KNEE_LOW:
#     qpos = np.copy(env.sim.qpos())
#     if qpos[10] < ACHILLES_ROD_LOW:
#         print(qpos[2])
    
    jpos[2] -= DELTA
    jpos[7] -= DELTA
    
    jpos[3] -= DELTA
    jpos[8] -= DELTA
    
    env.set_joint_pos(jpos, fbpos, iters=ITERS)
    env.render()

%matplotlib notebook

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

ax.axis("equal")

data = []
X = []
Y = []
Z = []

pitch = PITCH_LOW
while pitch < PITCH_HIGH:
    pitch += DELTA
    
    knee = KNEE_LOW
    while knee < KNEE_HIGH:
            knee += DELTA
            
            jpos[2] = pitch
            jpos[7] = pitch

            jpos[3] = knee
            jpos[8] = knee

            env.set_joint_pos(jpos, fbpos, iters=ITERS)
            
            footpos = np.zeros(6)
            env.sim.foot_pos(footpos)
            
            qpos = np.copy(env.sim.qpos())
            qpos = qpos[7:21]
            
            # get XZ foot position relative to center of mass
            footpos_relative = [footpos[0] - fbpos[0], footpos[2] - fbpos[2]]#,
                                #footpos[3] - fbpos[0], footpos[5] - fbpos[2]]
            
            data += [(footpos_relative, qpos)]
                        
            X += [footpos[0]]
            Y += [footpos[1]]
            Z += [footpos[2]]
            
            #if qpos[10] + qpos[2] < ACHILLES_LIMIT:
            #    continue 
                
            env.render()

    ax.plot(X, Y, Z)    
    X = []
    Y = []
    Z = []
    
    # ramp knee angle back down
    while jpos[3] > KNEE_LOW:
        jpos[3] -= DELTA
        jpos[8] -= DELTA
        
        env.set_joint_pos(jpos, fbpos, iters=ITERS)
        env.render()
        
plt.show()

<IPython.core.display.Javascript object>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler


class MLP(nn.Module):
    def __init__(self,
                 num_inputs,
                 num_outputs,
                 nonlinearity="tanh"):

        super(MLP, self).__init__()

        hidden_dims = (256, 256)
        
        self.hidden_layers = nn.ModuleList()
        self.hidden_layers += [nn.Linear(num_inputs, hidden_dims[0])]
        for l in range(len(hidden_dims) - 1):
            in_dim = hidden_dims[l]
            out_dim = hidden_dims[l + 1]
            self.hidden_layers += [nn.Linear(in_dim, out_dim)]
        
        self.out = nn.Linear(hidden_dims[-1], num_outputs)
        
        if nonlinearity == "relu":
            self.nonlinearity = F.relu
        elif nonlinearity == "tanh":
            self.nonlinearity = torch.tanh
        else:
            raise NotImplementedError

    def forward(self, inputs):
        x = inputs
        for l in self.hidden_layers:
            x = self.nonlinearity(l(x))
        pred = self.out(x)
        
        return pred

x = torch.Tensor(np.array([e[0] for e in data]))
y = torch.Tensor(np.array([e[1] for e in data]))

model = MLP(2, 14)
    
criterion = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=1e-4)
for t in range(500):
    sampler = BatchSampler(
        SubsetRandomSampler(range(y.size(0))),
        batch_size=128,
        drop_last=True
    )

    for indices in sampler:
        indices = torch.LongTensor(indices)
        
        y_batch = y[indices]
        x_batch = x[indices]

        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x_batch)

        # Compute and print loss
        loss = criterion(y_pred, y_batch)
        
        if t % 10 == 0:
            print(t, loss.item())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


0 2059.77783203125
0 1875.3763427734375
0 1781.901611328125
0 1763.2960205078125
0 1784.6451416015625
0 1597.506591796875
0 1529.0186767578125
0 1559.0750732421875
0 1468.08740234375
0 1352.6917724609375
0 1245.9921875
0 1303.6986083984375
0 1178.3961181640625
0 1159.2642822265625
0 1167.513427734375
0 920.8930053710938
10 75.02274322509766
10 80.81571197509766
10 65.8833236694336
10 78.90169525146484
10 62.889137268066406
10 74.34900665283203
10 69.75053405761719
10 68.86475372314453
10 72.82371520996094
10 69.81171417236328
10 65.59481048583984
10 56.83085632324219
10 71.7177505493164
10 67.5267562866211
10 63.164791107177734
10 62.89249038696289
20 45.063873291015625
20 40.43049240112305
20 45.36805725097656
20 38.22108459472656
20 51.81657028198242
20 47.203399658203125
20 40.78886032104492
20 41.03219223022461
20 39.22687911987305
20 38.91531753540039
20 39.369667053222656
20 49.26686477661133
20 41.6850471496582
20 43.80085754394531
20 53.68789291381836
20 46.881690979003906
30 4

250 1.8103317022323608
250 1.3709629774093628
250 1.2943665981292725
250 1.6649657487869263
250 1.448927402496338
250 1.7574697732925415
250 1.4135831594467163
250 1.2763285636901855
250 1.7351340055465698
250 1.4624814987182617
250 1.507819652557373
250 1.820220947265625
250 1.447758674621582
250 1.9443974494934082
250 2.1468863487243652
250 1.605892300605774
260 1.753088355064392
260 1.742143988609314
260 1.5417218208312988
260 1.688759207725525
260 1.797431468963623
260 1.5503655672073364
260 1.4538401365280151
260 1.5297341346740723
260 1.5412181615829468
260 1.66161048412323
260 1.2494726181030273
260 2.011862277984619
260 1.4736140966415405
260 1.4937700033187866
260 1.339125633239746
260 1.5047507286071777
270 1.4834991693496704
270 1.7181986570358276
270 1.4333605766296387
270 1.6511611938476562
270 1.5546350479125977
270 1.532326340675354
270 1.6133570671081543
270 1.5975900888442993
270 1.4830081462860107
270 1.2539176940917969
270 1.686476230621338
270 1.3661187887191772
270

480 0.8725531697273254
480 1.078678846359253
480 0.7458561062812805
480 1.0919365882873535
480 0.7760576605796814
480 1.0683435201644897
480 0.913625180721283
480 0.8096869587898254
480 0.8233713507652283
480 1.0410507917404175
480 0.7769317030906677
480 1.018959879875183
480 0.7128019332885742
480 0.8981271386146545
480 0.9558817148208618
480 0.8516659140586853
490 1.1430680751800537
490 0.8016284108161926
490 0.8896602392196655
490 0.7257093191146851
490 0.6538535952568054
490 0.9999331831932068
490 0.7548487782478333
490 0.8359929323196411
490 0.853980541229248
490 1.0954501628875732
490 0.9206574559211731
490 0.8475334048271179
490 1.0364184379577637
490 0.6289076209068298
490 0.8021847605705261
490 0.7374675273895264


In [7]:
import pickle
from cassie.cassiemujoco import pd_in_t

with open("bslip_traj.pkl", "rb") as f:
    bslip_data = pickle.load(f)

for i in range(200):
    fpos_des = torch.Tensor(bslip_data["foot"][i])
    
    lfoot = fpos_des[0:2]
    rfoot = fpos_des[2:4]

    qposl = model(lfoot).detach().numpy()
    qposr = model(rfoot).detach().numpy()

    print(fpos_des)
    #qpos = qpos.detach().numpy()
    
    qpos = np.concatenate([fbpos, qposl, qposr])

    qpos[0] = bslip_data["com"][i][0]
    qpos[2] = bslip_data["com"][i][1] + 0.1


    # fbpos[2] += 0.5

    # fbpos[3:7] = [0.25, 0.25, 0.25, 0.25]

    # qpos[0:7] = fbpos

    env.sim.set_qpos(qpos)
    env.sim.step_pd(pd_in_t())
    env.render()
    
    time.sleep(0.5)

tensor([-0.0139, -0.9926, -0.0000, -0.9960])
tensor([-0.0335, -0.9522, -0.0000, -0.9852])
tensor([-0.0481, -0.9074, -0.0000, -0.9711])
tensor([-0.0570, -0.8934, -0.0001, -0.9573])
tensor([-0.0609, -0.9015, -0.0001, -0.9438])
tensor([-0.0651, -0.9177, -0.0002, -0.9340])
tensor([-0.0677, -0.9239, -0.0003, -0.9239])
tensor([-0.0680, -0.9166, -0.0007, -0.9171])
tensor([-0.0689, -0.9265, -0.0017, -0.9268])
tensor([-0.0704, -0.9520, -0.0033, -0.9520])
tensor([-0.0726, -0.9847, -0.0055, -0.9847])
tensor([-0.0755, -0.9990, -0.0069, -0.9957])
tensor([-0.0781, -0.9985, -0.0083, -0.9852])
tensor([-0.0805, -0.9983, -0.0102, -0.9347])
tensor([-0.0828, -0.9982, -0.0128, -0.8978])
tensor([-0.0851, -0.9980, -0.0157, -0.8898])
tensor([-0.0871, -0.9978, -0.0190, -0.8946])
tensor([-0.0888, -0.9850, -0.0234, -0.8997])
tensor([-0.0916, -0.9566, -0.0257, -0.9021])
tensor([-0.0958, -0.9331, -0.0141, -0.9031])
tensor([-0.1020, -0.9197,  0.0002, -0.9061])
tensor([-0.1105, -0.9174,  0.0167, -0.9158])
tensor([-0

tensor([-0.0166, -0.9703,  0.0050, -0.8994])
tensor([-0.0312, -0.9705,  0.0611, -0.9018])
tensor([-0.0460, -0.9701,  0.1202, -0.9046])
tensor([-0.0613, -0.9693,  0.1504, -0.9109])
tensor([-0.0771, -0.9681,  0.1465, -0.9175])
tensor([-0.0937, -0.9665,  0.1459, -0.9256])
tensor([-0.1111, -0.9645,  0.1446, -0.9380])
tensor([-0.1293, -0.9620,  0.1363, -0.9620])
tensor([-0.1478, -0.9618,  0.1180, -0.9620])
tensor([-0.1643, -0.9614,  0.1002, -0.9632])
tensor([-0.1787, -0.9531,  0.0832, -0.9638])
tensor([-0.1933, -0.9405,  0.0671, -0.9644])
tensor([-0.2067, -0.9223,  0.0516, -0.9651])
tensor([-0.1876, -0.9067,  0.0366, -0.9661])
tensor([-0.1339, -0.8996,  0.0219, -0.9674])
tensor([-0.0831, -0.8971,  0.0074, -0.9688])
tensor([-0.0310, -0.8981, -0.0070, -0.9698])


### 